# Monitoring Models


 
> [!NOTE] Must use Python 3.10 SDK V2 for this demo.

In [ ]:
# Install required packages
%pip install scikit-learn

## Train a model

In [1]:
# Handle to the workspace
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
import mlflow

ml_client = MLClient.from_config(
    DefaultAzureCredential()
)

Found the config file in: /config.json


In [2]:
# Gather MLflow URI information from workspace
azureml_mlflow_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri
mlflow.set_tracking_uri(azureml_mlflow_uri)

In [3]:
# Import python packages
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, roc_curve
import pandas as pd
import numpy as np
import joblib
import os

Set the experiment Name

In [4]:
experiment_name = "Monitoring-Models-Experiment"
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='', creation_time=1670393486970, experiment_id='f2954ed9-3d7a-46a9-a891-c008bf426cea', last_update_time=None, lifecycle_stage='active', name='Monitoring-Models-Experiment', tags={}>

Upload data to datastore, and register it

In [5]:
data_src = "./data/diabetes-data"
os.makedirs(data_src, exist_ok=True)

model_path = "./models/monitoring"
os.makedirs(model_path, exist_ok=True)

In [6]:
# Uncomment to download data files

!curl --output-dir "./data/diabetes-data/" -O https://raw.githubusercontent.com/MicrosoftLearning/mslearn-dp100/main/data/diabetes.csv 
!curl --output-dir "./data/diabetes-data/" -O https://raw.githubusercontent.com/MicrosoftLearning/mslearn-dp100/main/data/diabetes2.csv 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  505k  100  505k    0     0  1779k      0 --:--:-- --:--:-- --:--:-- 1780k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  252k  100  252k    0     0   824k      0 --:--:-- --:--:-- --:--:--  823k


Start logging with MLflow

In [7]:
# start Logging
mlflow.start_run()

# enable autologging
# mlflow.sklearn.autolog()

# read csv data from local directory
data_set = pd.read_csv(data_src + "/diabetes.csv")

diabetes = data_set

#breaking up data into input/target features
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

#breaking data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

#training a model:
model = DecisionTreeClassifier().fit(X_train, y_train)

#calculating performance and logging them
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
mlflow.log_metric('Accuracy', float(acc))

y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
mlflow.log_metric('AUC', float(auc))

# registering the model to the workspace
print("Registering the model via MLFlow")
mlflow.sklearn.log_model(
    sk_model=model,
    registered_model_name="monitoring-diabetes-clr-mdl",
    artifact_path="monitoring-diabetes-clr-mdl",
)

# saving the model to a file
mlflow.sklearn.save_model(
    sk_model=model, 
    path=model_path,
)

# stop logging
mlflow.end_run()

Registering the model via MLFlow


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'monitoring-diabetes-clr-mdl' already exists. Creating a new version of this model...
2022/12/28 19:06:43 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: monitoring-diabetes-clr-mdl, version 4
Created version '4' of model 'monitoring-diabetes-clr-mdl'.


## Deploy a Model

You can check the **Models** page on Azure ML studio, to identify the latest version of your registered model. 

<img title="ws-model-registered" src="Media/ws-model-registered.png">

Alternatively, the code below will retrieve the latest version number for you to use.

In [11]:
# name the model you registered earlier in the training script
registered_model_name = "monitoring-diabetes-clr-mdl"

# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)

Create an Online endpoint

In [13]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
)

Configure the endpoint

In [14]:
# Creating a unique online endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "clr-diabetes-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
  name=online_endpoint_name,
  description="This is a diabetes classifier online endpoint",
  auth_mode="key",
  tags={
      "training_dataset": "diabetes-data",
      "model_type": "sklearn.DecisionTreeClassifier",
  },
)

endpoint = ml_client.begin_create_or_update(endpoint)

> [!NOTE] Make sure to wait for a notification that the endpoint has completed deployment.

<img title="Endpoint Deployment" src="Media/endpoint-deployment-succeeded.png">

<img title="Endpoint Deployment" src="Media/endpoint-deployment-succeeded_notification.png">

### Deploy the model to the endpoint

Once the endpoint is created, deploy the model with the entry script. Each endpoint can have multiple deployments. Direct traffic to these deployments can be specified using rules. Here you'll create a single deployment that handles 100% of the incoming traffic. We have chosen a color name for the deployment, for example, *blue*, *green*, *red* deployments, which is arbitrary.
Deploy the latest version of the model.  

> [!NOTE]
> Replace the endpoint name with the endpoint configured in the previous step.
> Expect this deployment to take approximately 6 to 8 minutes.

Create a red deployment


In [16]:
import time

model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

red_deployment = ManagedOnlineDeployment(
    name="red",
    #Replace the endpoint name below
    endpoint_name="clr-diabetes-12282034411976",
    model=model,
    instance_type="Standard_F4s_v2",
    instance_count=1,
    app_insights_enabled=True,
)

try:
    ml_client.online_deployments.begin_create_or_update(red_deployment)
    print("Creating red deployment on endpoint")
except Exception as e:
    print(e)
    print("Waiting 3 Minutes...")
    time.sleep(180)
    ml_client.online_deployments.begin_create_or_update(red_deployment)
    print("Creating red deployment on endpoint")


Check: endpoint clr-diabetes-12282034411976 exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


> [!Note] You can view the current provision status on the red deployment on the **Endpoint** Tab on the right hand side. 

Remember to wait for the deployment to be completed before moving to the next cell.

<img title="Endpoint Provision Pending Status" src="Media/endpoint-provision-status-pending.png">


Managing endpoints and deployments

In [17]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'red': 0}
https://clr-diabetes-12282034411976.southcentralus.inference.ml.azure.com/score


Now to increase the traffic to our new red deployment.

In [18]:
endpoint.traffic = {"red": 100}

Wait until both the endpoint and deployment are in the provisioned state. Then run the next cell.

In [19]:
ml_client.begin_create_or_update(endpoint)

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


When completed, it show look like this.

<img title="Endpoint Provision Status" src="Media/endpoint-provision-status.png">

## Use the Model

Make sure to capture the API key from the **Endpoint** '**Consume**' page. Replace it with the code below for api_key


<img title="Endpoint Details" src="Media/endpoint-details-page.png">



In [20]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script

#generating new data to use in querying
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
        [4,96,83,26,34,52.94533137,0.160199188,53],
        [1,125,83,41,235,19.65795152,0.150529189,23],
        [3,106,83,39,223,31.77645097,0.877332438,22],
        [0,148,58,11,179,39.19207553,0.160829008,45]]

data =  {"input_data": x_new}

body = str.encode(json.dumps(data))

url = endpoint.scoring_uri
api_key = 'PYcl229pEedyj8YtZRqu4QJW7rxnpoaF' # Replace this with the API key for the web service

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'red' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read()
    predicted_classes = json.loads(result.decode())
    for i in range(len(x_new)):
        print ("Patient {}".format(x_new[i]), predicted_classes[i] )

except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

Patient [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] 1
Patient [4, 96, 83, 26, 34, 52.94533137, 0.160199188, 53] 1
Patient [1, 125, 83, 41, 235, 19.65795152, 0.150529189, 23] 0
Patient [3, 106, 83, 39, 223, 31.77645097, 0.877332438, 22] 1
Patient [0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45] 0


## Conclusion

Now, we can navigate back to the Azure Portal, and look at the corresponding results in Application Insights.

<img src="Media/endpoint-deployment-metrics.png">


Or be sure to look for the Applications Insight instance that was created along side the Azure Machine Learning Workspace as an alternative.

<img src="Media/azure-app-insight.png">

To view the Transactions, click on the **Investigate** category for more details.

<img src="Media/azure-app-insight-transaction.png">

Try selecting one of the results for more details.

<img src="Media/azure-app-insight-transaction-results.png">

View the logs by selecting the desired operation within the **Performance** tab.

<img src="Media/azure-app-insight-performance.png">

Check out the query results from Azure Log Analytics

<img src="Media/logs-kql-commands.png">